<a href="https://colab.research.google.com/github/arcila-heisman/ml_introductory_labs/blob/master/Lab_5/Laboratorio_5_parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio 5 - Parte 2

### Máquinas de Vectores de Soporte

### 2019-I

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co

Modificado por Heisman Arcila. (Versión original en: https://github.com/jdariasl/ML_IntroductoryCourse/tree/master/Labs)

## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

#### Primer Integrante: Melissa Barba
#### C.C:
#### Segundo Integrante: Carlos Daniel Montoya Hurtado
#### C.C: 1017264995

In [1]:
from __future__ import division
import numpy as np
import math
import matplotlib.pyplot as plt
#Algunas advertencias que queremos evitar
import warnings
import scipy as sc
warnings.filterwarnings("ignore")

## Ejercicio 1

En este ejercicio usaremos la regresión por vectores de soporte para resolver el problema de regresión de la base de datos AirQuality (https://archive.ics.uci.edu/ml/datasets/Air+Quality).

La siguiente celda de código limpia la base de datos de todos sus datos faltantes y la deja lista en la variable DataBase.

In [2]:
#cargamos la bd de entrenamiento
db = np.loadtxt('BDatos/AirQuality.data',delimiter='\t')  # Assuming tab-delimiter
print("Dim de la base de datos original: " + str(np.shape(db)))
db = db.reshape(9357,13)
DataBase = db
#La base d edatos AirQuality tiene muchos valores perdidos, vamos a quitar todos esos registros de la base de datos
#Están marcados como -200. Donde haya un valor -200 eliminaremos el registro.
j = 0
for i in range(0,np.size(db,0)):
    if -200 == db[i,12]:
        #print i
        j+=1
        DataBase = np.delete(DataBase,i,0)
    
print("\nHay " + str(j) + " valores perdidos en la variable de salida. :(")

print("\nDim de la base de datos sin las muestras con variable de salida perdido "+ str(np.shape(DataBase)))

#Ya hemos eliminado los registros con valor de la variable de salida perdido.

#Ahora vamos a imputar los valores perdidos en cada una de las características
print("\nProcesando imputación de valores perdidos en las características . . .\n")

#data = open("AirQuality_cleaned.data", "w")

for k in range(0,np.size(DataBase,0)):
    for w in range(0,13):
        if -200 == DataBase[k,w]:
            DataBase[k,w] = round(np.mean(DataBase[:,w]))
        
    #data.write(str(DataBase[k,:]))

#data.close()
print("Imputación finalizada.\n")

hay_missed_values = False
for i in range(0,np.size(DataBase,0)):
    if -200 in DataBase[i,:]:
        hay_missed_values = True
if(hay_missed_values):
    print("Hay valores perdidos")
else:
    print("No hay valores perdidos en la base de datos. Ahora se puede procesar. La base de datos está en la variable DataBase")

Dim de la base de datos original: (9357, 13)

Hay 366 valores perdidos en la variable de salida. :(

Dim de la base de datos sin las muestras con variable de salida perdido (8991, 13)

Procesando imputación de valores perdidos en las características . . .

Imputación finalizada.

No hay valores perdidos en la base de datos. Ahora se puede procesar. La base de datos está en la variable DataBase


In [3]:
X = DataBase[:,0:12]

Y = DataBase[:,12]

Definimos la función

In [4]:
#Mean Absolute Percentage Error para los problemas de regresión
def MAPE(Y_est,Y):
    ind = Y != 0 #Remueve los elementos que son cero en la variable deseada
    N = np.size(Y[ind])
    mape = np.sum(abs((Y_est[ind].reshape(N,1) - Y[ind].reshape(N,1))/(Y[ind].reshape(N,1)+np.finfo(np.float).eps)))/N
    return mape 

Complete el código siguiente para usar la regresión por vectores de suporte usando la librería sklearn. https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html

In [5]:
from sklearn.model_selection import KFold
from sklearn import preprocessing
from sklearn.svm import SVR
import time
tiempo_i = time.time()

#Complete el código para crear el modelo SVM para regresión. 
#Use un kernel rbf con una malla de valores así: C en {0.1, 100} y gamma en {0.0001, 0.1}

#Implemetamos la metodología de validación cross validation con 10 folds
Folds = 5
Errores = np.ones(Folds)
j = 0
kf = KFold(n_splits=Folds)
vectorSupport = np.ones(Folds)
for train_index, test_index in kf.split(X):
    
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = Y[train_index], Y[test_index]  
    
     #Normalizamos los datos
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)
    
    #Complete el código
    modelo = SVR(kernel='rbf',C=100, gamma=1)
    
    
    # Entrenar el modelo
    modelo.fit(X_train,y_train)
    vectors = np.size(modelo.support_vectors_,0)
    vectorSupport[j] = vectors/len(X_train)
    # Validación del modelo
    ypred = modelo.predict(X_test)
 
    Errores[j] = MAPE(ypred, y_test)
    j+=1
    
print("\nError de validación: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))
print(" Porcentaje de Vectores de Soporte: " + str(np.mean(vectorSupport)))
print (("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i)) + " segundos.")


Error de validación: 0.2259390395748797 +/- 0.11094914643434282
 Porcentaje de Vectores de Soporte: 0.21910801302774435


Tiempo total de ejecución: 6.1095569133758545 segundos.


## Ejercicio 2

Una vez complete el código, realice las simulaciones necesarias para llenar la tabla siguiente:

In [10]:
import pandas as pd
# import qgrid
df_types = pd.DataFrame({
    'Kernel' : pd.Series(['linear','linear','linear','linear','linear','linear','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})
df_types["MAPE Promedio"] = ""
df_types["Intervalo de confianza"] = ""
df_types["% de Vectores de Soporte"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)
#Escriba acá su código

for i in df_types.index:
    print("Experimento: ", i)
    mape, error, psv = experimento2(i[0], i[1], i[2])
    df_types["MAPE Promedio"][i] = mape
    df_types["Intervalo de confianza"][i] = error
    df_types["% de Vectores de Soporte"][i] = psv
    


# Valores de referencia
# df_types["MAPE Promedio"][23] = "0.2259"
# df_types["Intervalo de confianza"][23] = "0.1109"
# df_types["% de Vectores de Soporte"][23] = "0.2191"

# qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
# qgrid_widget

Experimento:  ('linear', 0.001, 0.0)

Error de validación: 0.2869582909645992 +/- 0.15071982828475336
 Porcentaje de Vectores de Soporte: 1.0


Tiempo total de ejecución: 26.66771411895752 segundos.
Experimento:  ('linear', 0.01, 0.0)

Error de validación: 0.2869582909645992 +/- 0.15071982828475336
 Porcentaje de Vectores de Soporte: 1.0


Tiempo total de ejecución: 27.4910671710968 segundos.
Experimento:  ('linear', 0.1, 0.0)

Error de validación: 0.2869582909645992 +/- 0.15071982828475336
 Porcentaje de Vectores de Soporte: 1.0


Tiempo total de ejecución: 27.334622383117676 segundos.
Experimento:  ('linear', 1.0, 0.0)

Error de validación: 0.2869582909645992 +/- 0.15071982828475336
 Porcentaje de Vectores de Soporte: 1.0


Tiempo total de ejecución: 27.544062852859497 segundos.
Experimento:  ('linear', 10.0, 0.0)

Error de validación: 0.2869582909645992 +/- 0.15071982828475336
 Porcentaje de Vectores de Soporte: 1.0


Tiempo total de ejecución: 27.548644304275513 segundos.
Experimen

In [11]:
df_types

MAPE Promedio Intervalo de confianza  \
Kernel C       gamma                                        
linear 0.001   0.00       0.286958                0.15072   
       0.010   0.00       0.286958                0.15072   
       0.100   0.00       0.286958                0.15072   
       1.000   0.00       0.286958                0.15072   
       10.000  0.00       0.286958                0.15072   
       100.000 0.00       0.286958                0.15072   
rbf    0.001   0.01       0.204431               0.187421   
               0.10       0.191668               0.140413   
               1.00       0.237877               0.129976   
       0.010   0.01       0.204431               0.187421   
               0.10       0.191668               0.140413   
               1.00       0.237877               0.129976   
       0.100   0.01       0.204431               0.187421   
               0.10       0.191668               0.140413   
               1.00       0.237877               0.129976   
       1.000   0.01       0.204431               0.187421   
               0.10       0.191668               0.140413   
               1.00       0.237877               0.129976   
       10.000  0.01       0.204431               0.187421   
               0.10       0.191668               0.140413   
               1.00       0.237877               0.129976   
       100.000 0.01       0.204431               0.187421   
               0.10       0.191668               0.140413   
               1.00       0.237877               0.129976   

                     % de Vectores de Soporte  
Kernel C       gamma                           
linear 0.001   0.00                  0.194989  
       0.010   0.00                  0.194989  
       0.100   0.00                  0.194989  
       1.000   0.00                  0.194989  
       10.000  0.00                  0.194989  
       100.000 0.00                  0.194989  
rbf    0.001   0.01                 0.0625969  
               0.10                 0.0832052  
               1.00                   0.10848  
       0.010   0.01                 0.0625969  
               0.10                 0.0832052  
               1.00                   0.10848  
       0.100   0.01                 0.0625969  
               0.10                 0.0832052  
               1.00                   0.10848  
       1.000   0.01                 0.0625969  
               0.10                 0.0832052  
               1.00                   0.10848  
       10.000  0.01                 0.0625969  
               0.10                 0.0832052  
               1.00                   0.10848  
       100.000 0.01                 0.0625969  
               0.10                 0.0832052  
               1.00                   0.10848

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

2.1 ¿Cuál es la finalidad de usar las funciones kernel en el modelo SVM?

R/: Permite categorizar datos aunque de entrada no sean linealmente separables aplicando una transformación que los convierte en separables utilizando un hiperplano. 

2.2 ¿En este caso el porcentaje de vectores de soporte provee una información similar que en el problema de clasificación? Explique su respuesta.

R/: Las muestras que se determinan como vectores de soporte son las que se usan para definir el margen del modelo, que sirve para darle forma a la función. Luego, el porcentaje de vectores de soporte indica qué tan sobreajustado está el modelo o no, igual que en el caso de clasificación.


2.3 Realice una gráfica de las salidas reales vs las predicciones del modelo SVM, para evaluar visualmente el desempeño del mismo. Esto solo para la configuración en la cuál se encontró el menor error.

In [ ]:
'''Complete el código para hacer la gráfica aquí'''
plt.title('Real vs. Predicción')
plt.plot(y_test)
plt.plot(ypred)
plt.legend(['Real', 'Predicción'])

In [ ]:
# Experimento

In [ ]:
def experimento(kernel, C, gamma):
    tiempo_i = time.time()

    #Complete el código para crear el modelo SVM para regresión. 
    #Use un kernel rbf con una malla de valores así: C en {0.1, 100} y gamma en {0.0001, 0.1}
    if gamma == 0:
        clf = SVR(kernel, C)
    else:
        clf = SVR(kernel, C, gamma)

    #Implemetamos la metodología de validación cross validation con 10 folds
    Folds = 10
    psv_mean = np.zeros(10)
    Errores = np.ones(Folds)
    j = 0
    kf = KFold(n_splits=Folds)

    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

        #Complete el código
        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        # Entrenar el modelo
        modelo = clf.fit(X_train, y_train)


        # Validación del modelo
        ypred = modelo.predict(X_test)

        psv = modelo.support_vectors_
        psv_mean[j] = len(psv) / len(X_train)

        Errores[j] = MAPE(ypred, y_test)
        j+=1

    print("\nError de validación: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))

    print ("\n\nTiempo total de ejecución: ", str(time.time()-tiempo_i)), " segundos."
    mape = np.mean(Errores)
    error = np.std(Errores)
    psv = np.mean(psv_mean)
    
    return mape, error, psv


In [9]:
def experimento2(kernel, C, gamma):
    tiempo_i = time.time()

    #Complete el código para crear el modelo SVM para regresión. 
    #Use un kernel rbf con una malla de valores así: C en {0.1, 100} y gamma en {0.0001, 0.1}

    #Implemetamos la metodología de validación cross validation con 10 folds
    Folds = 5
    Errores = np.ones(Folds)
    j = 0
    kf = KFold(n_splits=Folds)
    vectorSupport = np.ones(Folds)
    psv_mean = np.zeros(10)
    for train_index, test_index in kf.split(X):

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = Y[train_index], Y[test_index]  

         #Normalizamos los datos
        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)

        #Complete el código
        
        if gamma == 0:
            modelo = SVR(kernel, C)
        else:
            modelo = SVR(kernel, C, gamma)
#         modelo = SVR(kernel,C,gamma)


        # Entrenar el modelo
        modelo.fit(X_train,y_train)
#         vectors = np.size(modelo.support_vectors_,0)
#         vectorSupport[j] = vectors/len(X_train)
        # Validación del modelo
        ypred = modelo.predict(X_test)
        
        psv = modelo.support_vectors_
        psv_mean[j] = len(psv) / len(X_train)

        Errores[j] = MAPE(ypred, y_test)
        j+=1

    print("\nError de validación: " + str(np.mean(Errores)) + " +/- " + str(np.std(Errores)))
    print(" Porcentaje de Vectores de Soporte: " + str(np.mean(vectorSupport)))
    print (("\n\nTiempo total de ejecución: " + str(time.time()-tiempo_i)) + " segundos.")
    
    mape = np.mean(Errores)
    error = np.std(Errores)
    psv = np.std(psv_mean)
    
    return mape, error, psv